# Setup

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# load data
df = pd.read_csv('/content/drive/MyDrive/Colab_Projects/AML_final_project/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv')
df.head()

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,is_metro_micro,total_beds_7_day_avg,all_adult_hospital_beds_7_day_avg,all_adult_hospital_inpatient_beds_7_day_avg,inpatient_beds_used_7_day_avg,all_adult_hospital_inpatient_bed_occupied_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg,inpatient_beds_7_day_avg,total_icu_beds_7_day_avg,total_staffed_adult_icu_beds_7_day_avg,icu_beds_used_7_day_avg,staffed_adult_icu_bed_occupancy_7_day_avg,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg,staffed_icu_adult_patients_confirmed_covid_7_day_avg,total_patients_hospitalized_confirmed_influenza_7_day_avg,icu_patients_confirmed_influenza_7_day_avg,total_patients_hospitalized_confirmed_influenza_and_covid_7_day_avg,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,...,icu_patients_confirmed_influenza_7_day_coverage,total_patients_hospitalized_confirmed_influenza_and_covid_7_day_coverage,previous_day_admission_adult_covid_confirmed_7_day_sum,previous_day_admission_adult_covid_confirmed_18-19_7_day_sum,previous_day_admission_adult_covid_confirmed_20-29_7_day_sum,previous_day_admission_adult_covid_confirmed_30-39_7_day_sum,previous_day_admission_adult_covid_confirmed_40-49_7_day_sum,previous_day_admission_adult_covid_confirmed_50-59_7_day_sum,previous_day_admission_adult_covid_confirmed_60-69_7_day_sum,previous_day_admission_adult_covid_confirmed_70-79_7_day_sum,previous_day_admission_adult_covid_confirmed_80+_7_day_sum,previous_day_admission_adult_covid_confirmed_unknown_7_day_sum,previous_day_admission_pediatric_covid_confirmed_7_day_sum,previous_day_covid_ED_visits_7_day_sum,previous_day_admission_adult_covid_suspected_7_day_sum,previous_day_admission_adult_covid_suspected_18-19_7_day_sum,previous_day_admission_adult_covid_suspected_20-29_7_day_sum,previous_day_admission_adult_covid_suspected_30-39_7_day_sum,previous_day_admission_adult_covid_suspected_40-49_7_day_sum,previous_day_admission_adult_covid_suspected_50-59_7_day_sum,previous_day_admission_adult_covid_suspected_60-69_7_day_sum,previous_day_admission_adult_covid_suspected_70-79_7_day_sum,previous_day_admission_adult_covid_suspected_80+_7_day_sum,previous_day_admission_adult_covid_suspected_unknown_7_day_sum,previous_day_admission_pediatric_covid_suspected_7_day_sum,previous_day_total_ED_visits_7_day_sum,previous_day_admission_influenza_confirmed_7_day_sum,geocoded_hospital_address,hhs_ids,previous_day_admission_adult_covid_confirmed_7_day_coverage,previous_day_admission_pediatric_covid_confirmed_7_day_coverage,previous_day_admission_adult_covid_suspected_7_day_coverage,previous_day_admission_pediatric_covid_suspected_7_day_coverage,previous_week_personnel_covid_vaccinated_doses_administered_7_day,total_personnel_covid_vaccinated_doses_none_7_day,total_personnel_covid_vaccinated_doses_one_7_day,total_personnel_covid_vaccinated_doses_all_7_day,previous_week_patients_covid_vaccinated_doses_one_7_day,previous_week_patients_covid_vaccinated_doses_all_7_day,is_corrected
0,aab2bb3ab769da90baf57242c96ec481afb5ec6a233784...,2021/08/06,LA,NaN,Crescent City Surgical Centre,NaN,NaN,70118.0,Short Term,NaN,False,10.0,10.0,10.0,4.9,4.9,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,70.0,70.0,34.0,34.0,0.0,0.0,0.0,0.0,70.0,...,7,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

# Preparing the Hospital Dataset

In [3]:
# omit unnecessary columns and rename
df = df[['collection_week', 'fips_code', 'inpatient_beds_7_day_avg', 'inpatient_beds_used_7_day_avg', 'total_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg' ]]
df = df.rename(columns={'collection_week': 'week', 'fips_code': 'fips_code', 'inpatient_beds_7_day_avg': 'inp_beds', 'inpatient_beds_used_7_day_avg': 'inp_beds_used', 'total_icu_beds_7_day_avg': 'icu_beds', 'icu_beds_used_7_day_avg': 'icu_beds_used'})
df.head()

,week,fips_code,inp_beds,inp_beds_used,icu_beds,icu_beds_used
0,2021/08/06,NaN,10.0,4.9,0.0,0.0
1,2021/08/06,NaN,14.0,-999999.0,0.0,0.0
2,2021/08/06,NaN,0.0,0.0,0.0,0.0
3,2021/08/06,1001.0,88.0,70.4,6.0,6.0
4,2021/08/06,1003.0,58.0,42.3,5.0,5.0


In [4]:
# filter NaN values
df = df.dropna()

# filter rows with -999999.0 values (not enough data to build 7 day average)
df = df[df.inp_beds != -999999.0]
df = df[df.inp_beds_used != -999999.0]
df = df[df.icu_beds != -999999.0]
df = df[df.icu_beds_used != -999999.0]

df.head()

,week,fips_code,inp_beds,inp_beds_used,icu_beds,icu_beds_used
3,2021/08/06,1001.0,88.0,70.4,6.0,6.0
4,2021/08/06,1003.0,58.0,42.3,5.0,5.0
5,2021/08/06,1003.0,128.0,101.4,10.0,19.0
6,2021/08/06,1003.0,186.0,186.4,17.0,25.9
7,2021/08/06,1005.0,42.9,30.4,5.0,4.9


In [5]:
# now we create a county sum (takes ~ 20 min on google colab)
county_data = {'week': [], 'fips_code': [], 'inp_beds': [],
               'inp_beds_used': [], 'icu_beds': [], 'icu_beds_used': []}

for week in df.week.unique():
  for fips in df.loc[df.week == week, 'fips_code'].unique():
    df_week_fips = df[(df.week == week) & (df.fips_code == fips)]

    county_data['week'].append(week)
    county_data['fips_code'].append(fips)
    county_data['inp_beds'].append(df_week_fips.inp_beds.sum())
    county_data['inp_beds_used'].append(df_week_fips.inp_beds_used.sum())
    county_data['icu_beds'].append(df_week_fips.icu_beds.sum())
    county_data['icu_beds_used'].append(df_week_fips.icu_beds_used.sum())

df = pd.DataFrame(county_data)
df.head()

,week,fips_code,inp_beds,inp_beds_used,icu_beds,icu_beds_used
0,2021/08/06,1001.0,88.0,70.4,6.0,6.0
1,2021/08/06,1003.0,372.0,330.1,32.0,49.9
2,2021/08/06,1005.0,42.9,30.4,5.0,4.9
3,2021/08/06,1007.0,30.0,11.0,0.0,0.0
4,2021/08/06,1009.0,35.0,25.3,6.0,4.9


In [6]:
# calculate county ratios
df['used_bed_ratio'] = df['inp_beds_used']/df['inp_beds']
df['used_icu_ratio'] = df['icu_beds_used']/df['icu_beds']
df = df.drop(columns = ['inp_beds', 'inp_beds_used', 'icu_beds', 'icu_beds_used'])
df.head()

,week,fips_code,used_bed_ratio,used_icu_ratio
0,2021/08/06,1001.0,0.800000,1.000000
1,2021/08/06,1003.0,0.887366,1.559375
2,2021/08/06,1005.0,0.708625,0.980000
3,2021/08/06,1007.0,0.366667,NaN
4,2021/08/06,1009.0,0.722857,0.816667


In [7]:
# 0 in the denominator creates NaN values. We exchange NaN values with a ratio of 1
df = df.fillna(1.)
df.head()

,week,fips_code,used_bed_ratio,used_icu_ratio
0,2021/08/06,1001.0,0.800000,1.000000
1,2021/08/06,1003.0,0.887366,1.559375
2,2021/08/06,1005.0,0.708625,0.980000
3,2021/08/06,1007.0,0.366667,1.000000
4,2021/08/06,1009.0,0.722857,0.816667


In [8]:
# remove ratios bigger than 1 since we dont know how to interpret them
df = df[(df.used_bed_ratio <= 1) & (df.used_icu_ratio <= 1)]
df.head()

,week,fips_code,used_bed_ratio,used_icu_ratio
0,2021/08/06,1001.0,0.800000,1.000000
2,2021/08/06,1005.0,0.708625,0.980000
3,2021/08/06,1007.0,0.366667,1.000000
4,2021/08/06,1009.0,0.722857,0.816667
5,2021/08/06,1011.0,0.443750,1.000000


In [9]:
# turn weekly format to monthly format
df['week'] = pd.to_datetime(df['week'], format = "%Y/%m/%d")
df['week'] = df['week'].dt.strftime('%Y-%m')
df = df.rename(columns = {'week' : 'month'})
df.head()

,month,fips_code,used_bed_ratio,used_icu_ratio
0,2021-08,1001.0,0.800000,1.000000
2,2021-08,1005.0,0.708625,0.980000
3,2021-08,1007.0,0.366667,1.000000
4,2021-08,1009.0,0.722857,0.816667
5,2021-08,1011.0,0.443750,1.000000


In [10]:
# average ratios over month (takes ~ 3 min on google colab)
monthly_data = {'month': [], 'fips_code': [], 'used_bed_ratio': [], 'used_icu_ratio': []}

for month in df.month.unique():
  for fips in df.loc[df.month == month, 'fips_code'].unique():
    df_month_fips = df[(df.month == month) & (df.fips_code == fips)]

    monthly_data['month'].append(month)
    monthly_data['fips_code'].append(fips)
    monthly_data['used_bed_ratio'].append(df_month_fips.used_bed_ratio.mean())
    monthly_data['used_icu_ratio'].append(df_month_fips.used_icu_ratio.mean())

df = pd.DataFrame(monthly_data)
df['fips_code'] = df['fips_code'].astype(int)

df.head()

,month,fips_code,used_bed_ratio,used_icu_ratio
0,2021-08,1001,0.800000,1.000000
1,2021-08,1005,0.708625,0.980000
2,2021-08,1007,0.366667,1.000000
3,2021-08,1009,0.722857,0.816667
4,2021-08,1011,0.443750,1.000000


# Combining Datasets

In [11]:
# loading data
df_hos = df
df_geo = pd.read_csv('/content/drive/MyDrive/Colab_Projects/AML_final_project/data_geo.csv')
df_geo.head()

,hosp_yn,death_yn,sex_female,age_0,age_18,age_50,age_65,race_native,race_asian,race_black,race_other,race_native_pacific,race_white,ethnicity_hispanic,case_month,state_fips_code,county_fips_code,case_onset_interval,symptom_status
0,False,False,True,False,True,False,False,False,True,False,False,False,False,True,2021-02,34,34003,0,True
1,False,False,True,False,True,False,False,False,False,False,False,False,True,True,2020-06,37,37107,0,True
2,False,False,True,False,True,False,False,False,False,False,True,False,False,True,2021-02,51,51700,0,True
3,False,False,True,True,False,False,False,False,False,False,False,False,True,True,2020-05,37,37163,0,True
4,False,False,True,False,True,False,False,False,False,False,False,False,True,True,2021-02,51,51540,0,True


In [28]:
# mapping month and fips code in the geo dataset to used_bed_ratio and used_icu_ratio in the hospital dataset
# first we create a dictionary to encode the mapping
mapping_bed = {}
mapping_icu = {} 

for month in df_geo.case_month.unique():
  mapping_bed[month] = {}
  mapping_icu[month] = {}

  for fips in df_geo[df_geo['case_month'] == month].county_fips_code.unique():
    used_bed_ratio = df_hos[(df_hos.month == month) & (df_hos.fips_code == fips)].used_bed_ratio.values
    used_icu_ratio = df_hos[(df_hos.month == month) & (df_hos.fips_code == fips)].used_icu_ratio.values

    if (len(used_bed_ratio) == 0) or (len(used_icu_ratio) == 0):
      mapping_bed[month][fips] = np.nan
      mapping_icu[month][fips] = np.nan

    else:
      mapping_bed[month][fips] = used_bed_ratio[0]
      mapping_icu[month][fips] = used_icu_ratio[0]


In [41]:
# now we use the dictionary for our mapping (takes ~ 2 min on google colab)
f_bed = lambda x: mapping_bed[x.case_month][x.county_fips_code]
f_icu = lambda x: mapping_icu[x.case_month][x.county_fips_code]

df_geo['used_bed_ratio'] = df_geo.loc[:, ['case_month', 'county_fips_code']].apply(f_bed, axis = 1)
df_geo['used_icu_ratio'] = df_geo.loc[:, ['case_month', 'county_fips_code']].apply(f_icu, axis = 1)

In [49]:
# drop values without ratios
df_geo = df_geo.dropna()
df_geo.head()

,hosp_yn,death_yn,sex_female,age_0,age_18,age_50,age_65,race_native,race_asian,race_black,race_other,race_native_pacific,race_white,ethnicity_hispanic,case_month,state_fips_code,county_fips_code,case_onset_interval,symptom_status,used_bed_ratio,used_icu_ratio
0,False,False,True,False,True,False,False,False,True,False,False,False,False,True,2021-02,34,34003,0,True,0.718141,0.451212
2,False,False,True,False,True,False,False,False,False,False,True,False,False,True,2021-02,51,51700,0,True,0.746529,0.774469
4,False,False,True,False,True,False,False,False,False,False,False,False,True,True,2021-02,51,51540,0,True,0.866410,0.755197
5,False,False,True,False,True,False,False,False,False,False,False,False,True,True,2020-08,36,36013,0,True,0.761232,0.659091
6,False,False,True,False,True,False,False,False,False,False,False,False,True,True,2020-12,39,39023,0,True,0.660607,0.855514


In [50]:
# print info
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3151446 entries, 0 to 3718690
Data columns (total 21 columns):
 #   Column               Dtype  
---  ------               -----  
 0   hosp_yn              bool   
 1   death_yn             bool   
 2   sex_female           bool   
 3   age_0                bool   
 4   age_18               bool   
 5   age_50               bool   
 6   age_65               bool   
 7   race_native          bool   
 8   race_asian           bool   
 9   race_black           bool   
 10  race_other           bool   
 11  race_native_pacific  bool   
 12  race_white           bool   
 13  ethnicity_hispanic   bool   
 14  case_month           object 
 15  state_fips_code      int64  
 16  county_fips_code     int64  
 17  case_onset_interval  int64  
 18  symptom_status       bool   
 19  used_bed_ratio       float64
 20  used_icu_ratio       float64
dtypes: bool(15), float64(2), int64(3), object(1)
memory usage: 213.4+ MB


In [52]:
# save data
df_geo.to_csv('/content/drive/MyDrive/Colab_Projects/AML_final_project/data_geo_hos.csv', index=False)